<a href="https://colab.research.google.com/github/linhvien/Stock-analysis/blob/main/Vietnamese_stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import libraries

In [1]:
from bs4 import BeautifulSoup as bs
import requests

import re
import json
import csv
from io import StringIO
from datetime import datetime, timedelta
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Overview

In [28]:
start_date = '2019-01-01'
end_date = str(datetime.now().strftime('%Y-%m-%d'))
stock = 'VTV'
portfolio = ''
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

#Financial data

Loading financial data of Vietnamese stocks from api vndirect

In [3]:
class Financialdata:
  url_BS = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=1,89,101,411&fromDate={}&toDate={}'
  url_IS = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=2,90,102,412&fromDate={}&toDate={}'
  url_CF = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=3,91,103,413&fromDate={}&toDate={}'
  url_ratio = 'https://finfo-api.vndirect.com.vn/v4/ratios?q=code:{}~itemCode:53030,52005,51050,53021,52001,52002,54018,712010,712020,712030,712040~reportDate:{}'
  
  def __init__(self,stock,start_date,end_date,*arg, **karg):
    self.stock = stock
    self.start_date = start_date
    self.end_date = end_date
    self.session = requests.Session()

# Get balance sheet
  def get_BS(self):
    page = self.session.get(self.url_BS.format(self.stock,self.start_date,self.end_date), headers=headers)
    data = page.json()

    data_dates = {}
    for item in data['data']['hits']:
      item = item['_source']
      date = item['fiscalDate']
      itemName = item['itemName']
      numericValue = item['numericValue']
      if date not in data_dates:
        data_dates[date] = [[], []]
      else:
        if itemName not in data_dates[date][0]:
          data_dates[date][0].append(itemName)
          data_dates[date][1].append(numericValue)

    for i, (date, item) in enumerate(data_dates.items()):
      df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
      if i == 0:
        df = df_date
      else:
        df = pd.merge(df, df_date, how='inner')
    return df

# get income statement
  def get_IS(self):
    page = self.session.get(self.url_IS.format(self.stock,self.start_date,self.end_date), headers=headers)
    data = page.json()

    data_dates = {}
    for item in data['data']['hits']:
      item = item['_source']
      date = item['fiscalDate']
      itemName = item['itemName']
      numericValue = item['numericValue']
      if date not in data_dates:
        data_dates[date] = [[], []]
      else:
        if itemName not in data_dates[date][0]:
          data_dates[date][0].append(itemName)
          data_dates[date][1].append(numericValue)

    for i, (date, item) in enumerate(data_dates.items()):
      df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
      if i == 0:
        df = df_date
      else:
        df = pd.merge(df, df_date, how='inner')
    return df

# get cashflow statement
  def get_CF(self):
    page = self.session.get(self.url_CF.format(self.stock,self.start_date,self.end_date), headers=headers)
    data = page.json()

    data_dates = {}
    for item in data['data']['hits']:
      item = item['_source']
      date = item['fiscalDate']
      itemName = item['itemName']
      numericValue = item['numericValue']
      if date not in data_dates:
        data_dates[date] = [[], []]
      else:
        if itemName not in data_dates[date][0]:
          data_dates[date][0].append(itemName)
          data_dates[date][1].append(numericValue)

    for i, (date, item) in enumerate(data_dates.items()):
      df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
      if i == 0:
        df = df_date
      else:
        df = pd.merge(df, df_date, how='inner')
    return df

In [4]:
BS_data = Financialdata(stock,start_date,end_date).get_BS()
BS_data.to_csv(r'BS_{}.csv'.format(stock),index=False)
BS_data.head()

,index,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
0,Tiền và các khoản tương đương tiền,5624321231,3392269125,4595655269,18739125449,40099874588,49918374688,40964841863,11832161341,31923108344,31147125910
1,Tiền,5624321231,3392269125,4595655269,18739125449,40099874588,49918374688,40964841863,11832161341,31923108344,31147125910
2,Các khoản tương đương tiền,0,0,0,0,0,0,0,0,0,0
3,Các khoản đầu tư tài chính ngắn hạn,0,0,0,0,0,0,0,0,0,0
4,Đầu tư ngắn hạn,0,0,0,0,0,0,0,0,0,0


In [5]:
IS_data = Financialdata(stock,start_date,end_date).get_IS()
IS_data.to_csv(r'IS_{}.csv'.format(stock),index=False)
IS_data.head()

,index,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
0,Các khoản giảm trừ doanh thu,0,0,0,0,0,0,0,0,0,0
1,Doanh thu thuần,425469768708,356264901421,370102279537,358533856670,434745576917,337824896375,357393361441,516162708757,655680181772,640366355992
2,Giá vốn hàng bán,383290354050,319040167298,327743142961,323785192414,390605150840,300460217125,311738741888,456010741724,581196053338,565436557458
3,Lợi nhuận gộp,42179414658,37224734123,42359136576,34748664256,44140426077,37364679250,45654619553,60151967033,74484128434,74929798534
4,Doanh thu hoạt động tài chính,257154952,2424112,3119604,4804990,1640005940,48569556,121664735,355082983,267475917,274366328


In [6]:
CF_data = Financialdata(stock,start_date,end_date).get_CF()
CF_data.to_csv(r'CF_{}.csv'.format(stock),index=False)
CF_data.head()

,index,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
0,Chi phí khấu hao tài sản cố định,3031474297,3033645507,3033277726,3046833295,3060036210,3063586821,2868758280,2917196588,2984545577,2541628920
1,Phân bổ lợi thế thương mại,0,0,0,0,0,0,0,0,0,0
2,"Dự phòng giảm giá các khoản đầu tư ngắn hạn, d...",0,0,0,0,0,0,0,0,0,0
3,"Lãi, lỗ chênh lệch tỷ giá hối đoái chưa thực hiện",0,0,0,0,0,0,-1303130,2710752,-1667431,259809
4,Lãi/(lỗ) từ thanh lý tài sản cố định,0,0,0,0,0,0,0,0,0,0


#Industry analysis

In [9]:
# Get industry list from vietstock
r = requests.get('https://finance.vietstock.vn/chi-so-nganh.htm',headers=headers)
soup = bs(r.content)
def Getsectorname():
  sectors = soup.select('a[href^="/nganh"]')
  all_sector = []
  for i in sectors:
    sector_name = i.get('title')
    if sector_name not in all_sector:
      all_sector.append(sector_name)
  return all_sector

In [10]:
Getsectorname()

['Bán buôn',
 'Bảo hiểm',
 'Bất động sản',
 'Chứng khoán',
 'Công nghệ và thông tin',
 'Bán lẻ',
 'Chăm sóc sức khỏe',
 'Khai khoáng',
 'Ngân hàng',
 'Nông - Lâm - Ngư',
 'SX Thiết bị, máy móc',
 'SX Hàng gia dụng',
 'Sản phẩm cao su',
 'SX Nhựa - Hóa chất',
 'Thực phẩm - Đồ uống',
 'Chế biến Thủy sản',
 'Vật liệu xây dựng',
 'Tiện ích',
 'Vận tải - kho bãi',
 'Xây dựng',
 'Dịch vụ lưu trú, ăn uống, giải trí',
 'SX Phụ trợ',
 'Thiết bị điện',
 'Dịch vụ tư vấn, hỗ trợ',
 'Tài chính khác']

In [11]:
def GetURL():
    sectors = soup.select('a[href^="/nganh"]')
    all_sector_URL = []
    for sector in sectors:
        sector_ID = sector.get('href')
        sector_URL = 'https://finance.vietstock.vn' + sector_ID
        if sector_URL not in all_sector_URL:
            all_sector_URL.append(sector_URL)
    return all_sector_URL

In [17]:
url = GetURL()
url

['https://finance.vietstock.vn/nganh/1-ban-buon.htm',
 'https://finance.vietstock.vn/nganh/2-bao-hiem.htm',
 'https://finance.vietstock.vn/nganh/3-bat-dong-san.htm',
 'https://finance.vietstock.vn/nganh/5-chung-khoan.htm',
 'https://finance.vietstock.vn/nganh/6-cong-nghe-va-thong-tin.htm',
 'https://finance.vietstock.vn/nganh/7-ban-le.htm',
 'https://finance.vietstock.vn/nganh/8-cham-soc-suc-khoe.htm',
 'https://finance.vietstock.vn/nganh/10-khai-khoang.htm',
 'https://finance.vietstock.vn/nganh/11-ngan-hang.htm',
 'https://finance.vietstock.vn/nganh/12-nong-lam-ngu.htm',
 'https://finance.vietstock.vn/nganh/15-sx-thiet-bi-may-moc.htm',
 'https://finance.vietstock.vn/nganh/16-sx-hang-gia-dung.htm',
 'https://finance.vietstock.vn/nganh/17-san-pham-cao-su.htm',
 'https://finance.vietstock.vn/nganh/18-sx-nhua-hoa-chat.htm',
 'https://finance.vietstock.vn/nganh/19-thuc-pham-do-uong.htm',
 'https://finance.vietstock.vn/nganh/20-che-bien-thuy-san.htm',
 'https://finance.vietstock.vn/nganh/21

In [27]:
full_stock =[]
for i in url:
  industry = pd.read_html(requests.get(i,headers=headers).text)
  industry = pd.DataFrame(industry[1])
  full_stock.append(industry)
symbols = pd.concat(full_stock, axis=0, ignore_index=True)
symbols = symbols.drop(columns=['STT','Sàn','KLCPLH','Giá 1 ngày'])
symbols.tail()

,Mã CK,Ngành cấp 3,Giá 5 ngày
735,VCM,Dịch vụ việc làm,15300
736,VLA,Dịch vụ lập trình máy tính,17000
737,VNC,"Các dịch vụ chuyên môn, khoa học và kỹ thuật khác",32100
738,OGC,Các hoạt động đầu tư tài chính khác,6800
739,TVC,Các hoạt động đầu tư tài chính khác,17000


In [77]:
symbols['Ngành cấp 3']

0              Bán buôn quần áo, vải và vật tư liên quan
1      Bán buôn thực phẩm, tạp hóa và các sản phẩm có...
2                   Bán buôn dầu và các sản phẩm dầu khí
3      Bán buôn các thiết bị, vật tư chuyên môn và th...
4                   Bán buôn máy móc, thiết bị và vật tư
                             ...                        
735                                     Dịch vụ việc làm
736                           Dịch vụ lập trình máy tính
737    Các dịch vụ chuyên môn, khoa học và kỹ thuật khác
738                  Các hoạt động đầu tư tài chính khác
739                  Các hoạt động đầu tư tài chính khác
Name: Ngành cấp 3, Length: 740, dtype: object

In [83]:
stock = 'VIC'
industry = symbols.loc[symbols['Mã CK'] == stock, 'Ngành cấp 3'].to_string(index=False).strip()
industry

'Phát triển bất động sản'

In [ ]:
symbols.loc[symbols['Ngành cấp 3'] == industry]